# Crypto ZigZag ML - Batch Training All Symbols

Train models for 22 symbols × 2 timeframes (15m + 1h) on Google Colab

This will generate:
- 44 signal classification models (5-class)
- 44 signal detection models (binary)
- 44 signal type models (if enabled)
- Total: 132 production models


## Step 1: Setup


In [ ]:
import tensorflow as tf
print(f'GPU: {tf.config.list_physical_devices("GPU")}')

from google.colab import drive
drive.mount('/content/drive')

import os
os.makedirs('/content/drive/MyDrive/crypto-zigzag-ml/models', exist_ok=True)
os.makedirs('/content/drive/MyDrive/crypto-zigzag-ml/training_logs', exist_ok=True)
print('Directories ready')

In [ ]:
import sys
project_root = '/content/drive/MyDrive/crypto-zigzag-ml'
sys.path.insert(0, project_root)

import pandas as pd
import numpy as np
import pickle
import json
from datetime import datetime
from tensorflow import keras
from tensorflow.keras import layers

from data.fetch_data import CryptoDataFetcher
from src.zigzag_indicator import ZigZagIndicator
from src.features import FeatureEngineer
from src.utils import time_series_split

print('All imports successful')

## Step 2: Define Symbols and Timeframes


In [ ]:
# 22 symbols as specified
SYMBOLS = [
    'BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'XRPUSDT', 'ADAUSDT',
    'DOGEUSDT', 'MATICUSDT', 'LINKUSDT', 'LITUSDT', 'UNIUSDT',
    'AVAXUSDT', 'SOLUUSDT', 'FTMUSDT', 'AAVEUSDT', 'CRVUSDT',
    'MKRUSDT', 'SNXUSDT', 'COMPUSDT', 'LRCUSDT', 'GRTUSDT',
    'ALGOUSDT', 'ATOMUSDT'
]

TIMEFRAMES = ['15m', '1h']

print(f'Total configurations: {len(SYMBOLS)} symbols × {len(TIMEFRAMES)} timeframes = {len(SYMBOLS) * len(TIMEFRAMES)} models')
print(f'With 3 models per config: {len(SYMBOLS) * len(TIMEFRAMES) * 3} total models')

## Step 3: Model Builder Function


In [ ]:
def build_classification_model(input_shape):
    """Build 5-class signal classification model"""
    inputs = layers.Input(shape=input_shape)
    
    x = layers.LSTM(256, return_sequences=True)(inputs)
    x = layers.Dropout(0.3)(x)
    x = layers.LSTM(128, return_sequences=False)(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    
    output = layers.Dense(5, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=output)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0005),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

def build_detection_model(input_shape):
    """Build binary signal detection model"""
    inputs = layers.Input(shape=input_shape)
    
    x = layers.LSTM(128, return_sequences=True)(inputs)
    x = layers.Dropout(0.2)(x)
    x = layers.LSTM(64, return_sequences=False)(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    
    output = layers.Dense(1, activation='sigmoid')(x)
    
    model = keras.Model(inputs=inputs, outputs=output)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

print('Model builders ready')

## Step 4: Training Function


In [ ]:
def train_symbol_timeframe(symbol, timeframe, fetcher, zigzag, fe):
    """Train models for one symbol+timeframe combination"""
    
    try:
        print(f'\n{'='*60}')
        print(f'Training: {symbol} {timeframe}')
        print(f'{'='*60}')
        
        # Fetch data
        print(f'Fetching data...')
        data = fetcher.fetch_symbol_timeframe(symbol, timeframe)
        
        if len(data) < 1000:
            print(f'  ⚠ Insufficient data ({len(data)} bars), skipping')
            return None
        
        print(f'  ✓ {len(data)} bars fetched')
        
        # Apply ZigZag
        print(f'Applying ZigZag...')
        data = zigzag.label_kbars(data)
        label_counts = data['zigzag_label'].value_counts()
        print(f'  ✓ Labels: {dict(label_counts)}'  )
        
        # Features
        print(f'Engineering features...')
        data = fe.calculate_all_features(data)
        feature_cols = fe.get_feature_columns(data)
        data[feature_cols] = data[feature_cols].fillna(method='ffill').fillna(0)
        print(f'  ✓ {len(feature_cols)} features')
        
        # Split
        train_df, val_df, test_df = time_series_split(data, 0.7, 0.15)
        
        # Prepare
        selected_features = feature_cols[:40]
        X_train = train_df[selected_features].values.astype(np.float32)
        y_train = train_df['zigzag_label'].values
        X_val = val_df[selected_features].values.astype(np.float32)
        y_val = val_df['zigzag_label'].values
        X_test = test_df[selected_features].values.astype(np.float32)
        y_test = test_df['zigzag_label'].values
        
        # Normalize
        mean = X_train.mean(axis=0)
        std = X_train.std(axis=0) + 1e-8
        X_train = (X_train - mean) / std
        X_val = (X_val - mean) / std
        X_test = (X_test - mean) / std
        
        # Sequences
        def create_seq(X, y):
            Xs, ys = [], []
            for i in range(len(X) - 20):
                Xs.append(X[i:i+20])
                ys.append(y[i+20])
            return np.array(Xs, dtype=np.float32), np.array(ys)
        
        X_train_seq, y_train_seq = create_seq(X_train, y_train)
        X_val_seq, y_val_seq = create_seq(X_val, y_val)
        X_test_seq, y_test_seq = create_seq(X_test, y_test)
        
        print(f'  ✓ Sequences: {X_train_seq.shape}')
        
        # Class weights
        unique, counts = np.unique(y_train_seq, return_counts=True)
        total = len(y_train_seq)
        class_weights = {}
        for u, c in zip(unique, counts):
            if u == 0:
                class_weights[u] = 1.0
            else:
                class_weights[u] = total / (5 * c) * 3
        
        # Binary labels
        y_train_binary = (y_train_seq != 0).astype(np.float32)
        y_val_binary = (y_val_seq != 0).astype(np.float32)
        y_test_binary = (y_test_seq != 0).astype(np.float32)
        
        # Train Classification Model
        print(f'Training classification model...')
        clf_model = build_classification_model((X_train_seq.shape[1], X_train_seq.shape[2]))
        
        early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, verbose=0)
        
        clf_model.fit(
            X_train_seq, y_train_seq,
            validation_data=(X_val_seq, y_val_seq),
            epochs=200,
            batch_size=32,
            class_weight=class_weights,
            callbacks=[early_stop],
            verbose=0
        )
        
        clf_loss, clf_acc = clf_model.evaluate(X_test_seq, y_test_seq, verbose=0)
        print(f'  ✓ Classification Test Acc: {clf_acc:.4f}')
        
        # Train Detection Model
        print(f'Training detection model...')
        det_model = build_detection_model((X_train_seq.shape[1], X_train_seq.shape[2]))
        
        det_model.fit(
            X_train_seq, y_train_binary,
            validation_data=(X_val_seq, y_val_binary),
            epochs=100,
            batch_size=32,
            callbacks=[early_stop],
            verbose=0
        )
        
        det_loss, det_acc = det_model.evaluate(X_test_seq, y_test_binary, verbose=0)
        print(f'  ✓ Detection Test Acc: {det_acc:.4f}')
        
        # Save models
        model_dir = f'{project_root}/models/{symbol.lower()}_{timeframe}'
        os.makedirs(model_dir, exist_ok=True)
        
        clf_model.save(f'{model_dir}/classification.h5')
        det_model.save(f'{model_dir}/detection.h5')
        
        # Save parameters
        params = {
            'mean': mean.tolist(),
            'std': std.tolist(),
            'class_weights': {int(k): v for k, v in class_weights.items()},
            'clf_acc': float(clf_acc),
            'det_acc': float(det_acc)
        }
        with open(f'{model_dir}/params.json', 'w') as f:
            json.dump(params, f, indent=2)
        
        print(f'  ✓ Models saved to {model_dir}')
        
        return {
            'symbol': symbol,
            'timeframe': timeframe,
            'clf_acc': clf_acc,
            'det_acc': det_acc,
            'status': 'success'
        }
    
    except Exception as e:
        print(f'  ✗ Error: {str(e)}')
        return {
            'symbol': symbol,
            'timeframe': timeframe,
            'status': 'failed',
            'error': str(e)
        }

print('Training function ready')

## Step 5: Batch Training


In [ ]:
fetcher = CryptoDataFetcher()
zigzag = ZigZagIndicator(depth=12, deviation=5, backstep=2)
fe = FeatureEngineer(lookback_periods=[5, 10, 20, 50, 200])

results = []

for symbol in SYMBOLS:
    for timeframe in TIMEFRAMES:
        result = train_symbol_timeframe(symbol, timeframe, fetcher, zigzag, fe)
        if result:
            results.append(result)

print(f'\n{'='*60}')
print('BATCH TRAINING COMPLETE')
print(f'{'='*60}')

## Step 6: Summary Report


In [ ]:
# Save results
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
results_file = f'{project_root}/training_logs/batch_results_{timestamp}.json'

with open(results_file, 'w') as f:
    json.dump(results, f, indent=2)

print(f'Results saved to: {results_file}')

# Summary
successful = [r for r in results if r['status'] == 'success']
failed = [r for r in results if r['status'] == 'failed']

print(f'\nSuccessful: {len(successful)}/{len(results)}')
print(f'Failed: {len(failed)}/{len(results)}')

if successful:
    avg_clf = np.mean([r['clf_acc'] for r in successful])
    avg_det = np.mean([r['det_acc'] for r in successful])
    print(f'\nAverage Classification Accuracy: {avg_clf:.4f}')
    print(f'Average Detection Accuracy: {avg_det:.4f}')
    
    print(f'\nTop 5 Classification Models:')
    sorted_clf = sorted(successful, key=lambda x: x['clf_acc'], reverse=True)[:5]
    for r in sorted_clf:
        print(f'  {r["symbol"]} {r["timeframe"]}: {r["clf_acc"]:.4f}')

if failed:
    print(f'\nFailed Models:')
    for r in failed:
        print(f'  {r["symbol"]} {r["timeframe"]}: {r.get("error", "Unknown error")}')

## Step 7: Verify Models


In [ ]:
import os

model_base = f'{project_root}/models'
model_dirs = [d for d in os.listdir(model_base) if os.path.isdir(os.path.join(model_base, d))]

print(f'Total model directories: {len(model_dirs)}')
print(f'Expected: {len(SYMBOLS) * len(TIMEFRAMES)} (22 × 2)')

# Check model files
complete = 0
for model_dir in model_dirs:
    path = os.path.join(model_base, model_dir)
    has_clf = os.path.exists(os.path.join(path, 'classification.h5'))
    has_det = os.path.exists(os.path.join(path, 'detection.h5'))
    has_params = os.path.exists(os.path.join(path, 'params.json'))
    
    if has_clf and has_det and has_params:
        complete += 1

print(f'\nComplete model sets: {complete}/{len(model_dirs)}')
print(f'Total model files: {complete * 3} (classification + detection + params)')

print(f'\nModel structure:')
print(f'📁 models/')
print(f'  ├── btcusdt_15m/')
print(f'  │   ├── classification.h5 (5-class signal classifier)')
print(f'  │   ├── detection.h5 (binary signal detector)')
print(f'  │   └── params.json (normalization params)')
print(f'  ├── btcusdt_1h/')
print(f'  │   └── ...')
print(f'  └── ... (22 symbols × 2 timeframes = 44 model sets)')

## Done!

All models are trained and saved to Google Drive.
You can now download them and deploy to production!
